In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import MinMaxScaler, StandardScaler, Normalizer, MaxAbsScaler
# from sklearn.pipeline import make_pipeline
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, ShuffleSplit, cross_val_score
import matplotlib.pyplot as plt
import seaborn as sns
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

!pip install -q xgboost
from xgboost import XGBRegressor

/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)


In [2]:
train = pd.read_csv('/kaggle/input/playground-series-s3e6/train.csv')
train = train.drop('id', axis=1)
print(train.shape)
train.head()

(22730, 17)


,squareMeters,numberOfRooms,hasYard,hasPool,floors,cityCode,cityPartRange,numPrevOwners,made,isNewBuilt,hasStormProtector,basement,attic,garage,hasStorageRoom,hasGuestRoom,price
0,34291,24,1,0,47,35693,2,1,2000,0,1,8,5196,369,0,3,3436795.2
1,95145,60,0,1,60,34773,1,4,2000,0,1,729,4496,277,0,6,9519958.0
2,92661,45,1,1,62,45457,4,8,2020,1,1,7473,8953,245,1,9,9276448.1
3,97184,99,0,0,59,15113,1,1,2000,0,1,6424,8522,256,1,9,9725732.2
4,61752,100,0,0,57,64245,8,4,2018,1,0,7151,2786,863,0,7,6181908.8


In [3]:
def prepare_data(data):
    sMq = np.quantile(data.squareMeters, 0.99)
    data.loc[data.squareMeters>sMq, 'squareMeters'] = sMq
    
    data['numRooms'] = 'E'
    data.loc[(data.numberOfRooms>20) & (data.numberOfRooms<=40), 'numRooms'] = 'D'
    data.loc[(data.numberOfRooms>40) & (data.numberOfRooms<=60), 'numRooms'] = 'C'
    data.loc[(data.numberOfRooms>60) & (data.numberOfRooms<=80), 'numRooms'] = 'B'
    data.loc[(data.numberOfRooms>80), 'numRooms'] = 'A'
    
    sF = np.quantile(data.floors, 0.99)
    data.loc[data.floors>sF, 'floors'] = sF
    
    data['numFloors'] = 'D'
    data.loc[(data.floors>25) & (data.floors<=50), 'numFloors'] = 'C'
    data.loc[(data.floors>50) & (data.floors<=75), 'numFloors'] = 'B'
    data.loc[(data.floors>75), 'numFloors'] = 'A'

    sM = np.quantile(data.made, 0.99)
    data.loc[data.made>sM, 'made'] = sM
    
    data['numMade'] = 'C'
    data.loc[(data.made>2000) & (data.made<=2010), 'numMade'] = 'B'
    data.loc[(data.made>2010), 'numMade'] = 'A'

    sB = np.quantile(data.basement, 0.99)
    data.loc[data.basement>sB, 'basement'] = sB
    
    data['numBasement'] = 'E'
    data.loc[(data.basement>2000) & (data.basement<=4000), 'numBasement'] = 'D'
    data.loc[(data.basement>4000) & (data.basement<=6000), 'numBasement'] = 'C'
    data.loc[(data.basement>6000) & (data.basement<=8000), 'numBasement'] = 'B'
    data.loc[(data.basement>8000), 'numBasement'] = 'A'

    sA = np.quantile(data.attic, 0.99)
    data.loc[data.attic>sA, 'attic'] = sA
    
    data['numAttic'] = 'E'
    data.loc[(data.attic>2000) & (data.attic<=4000), 'numAttic'] = 'D'
    data.loc[(data.attic>4000) & (data.attic<=6000), 'numAttic'] = 'C'
    data.loc[(data.attic>6000) & (data.attic<=8000), 'numAttic'] = 'B'
    data.loc[(data.attic>8000), 'numAttic'] = 'A'
    
    sG = np.quantile(data.garage, 0.99)
    data.loc[data.garage>sG, 'garage'] = sG
    
    data['numGarage'] = 'D'
    data.loc[(data.garage>250) & (data.garage<=500), 'numGarage'] = 'C'
    data.loc[(data.garage>500) & (data.garage<=750), 'numGarage'] = 'B'
    data.loc[(data.garage>750), 'numGarage'] = 'A'

    data = data.drop(columns=['numberOfRooms','floors','made','basement','attic','garage'])
    return data

In [4]:
train = prepare_data(train)

In [5]:
train.head()

,squareMeters,hasYard,hasPool,cityCode,cityPartRange,numPrevOwners,isNewBuilt,hasStormProtector,hasStorageRoom,hasGuestRoom,price,numRooms,numFloors,numMade,numBasement,numAttic,numGarage
0,34291,1,0,35693,2,1,0,1,0,3,3436795.2,D,C,C,E,C,C
1,95145,0,1,34773,1,4,0,1,0,6,9519958.0,C,B,C,E,C,C
2,92661,1,1,45457,4,8,1,1,1,9,9276448.1,C,B,A,B,A,D
3,97184,0,0,15113,1,1,0,1,1,9,9725732.2,A,B,C,B,A,C
4,61752,0,0,64245,8,4,1,0,0,7,6181908.8,A,B,A,B,D,A


In [6]:
num_col = ['squareMeters', 'cityCode', 'price']

In [7]:
cat_num = ['cityPartRange', 'numPrevOwners', 'hasGuestRoom', 'numRooms', 'numFloors','numMade', 'numBasement','numAttic', 'numGarage']
train = pd.get_dummies(data=train, columns=cat_num)
train.head()

,squareMeters,hasYard,hasPool,cityCode,isNewBuilt,hasStormProtector,hasStorageRoom,price,cityPartRange_1,cityPartRange_2,...,numBasement_E,numAttic_A,numAttic_B,numAttic_C,numAttic_D,numAttic_E,numGarage_A,numGarage_B,numGarage_C,numGarage_D
0,34291,1,0,35693,0,1,0,3436795.2,0,1,...,1,0,0,1,0,0,0,0,1,0
1,95145,0,1,34773,0,1,0,9519958.0,1,0,...,1,0,0,1,0,0,0,0,1,0
2,92661,1,1,45457,1,1,1,9276448.1,0,0,...,0,1,0,0,0,0,0,0,0,1
3,97184,0,0,15113,0,1,1,9725732.2,1,0,...,0,1,0,0,0,0,0,0,1,0
4,61752,0,0,64245,1,0,0,6181908.8,0,0,...,0,0,0,0,1,0,1,0,0,0


In [8]:
target = train.price
train = train.drop(['price'], axis=1)

In [9]:
x_train, x_test, y_train, y_test = train_test_split(train, target, test_size=0.3, random_state=15)

In [10]:
num_col.remove('price')

In [11]:
preproc = (StandardScaler(), MinMaxScaler(), Normalizer(), MaxAbsScaler())

tr_data = x_train
ts_data = x_test
min_ = 10**15

for p in preproc:
    tr_data[num_col] = p.fit_transform(tr_data[num_col])
    ts_data[num_col] = p.fit_transform(ts_data[num_col])
    xg = XGBRegressor()
    xg.fit(x_train, y_train)
    predict_pipe = xg.predict(x_test)
    print('***********************************************')
    print(f'{p}: {mean_squared_error(y_test, predict_pipe)}')
    print('***********************************************\n')
    if mean_squared_error(y_test, predict_pipe)<min_:
        min_ = mean_squared_error(y_test, predict_pipe)
        print(f'*{p}*')

***********************************************
StandardScaler(): 70632695765.34904
***********************************************

*StandardScaler()*
***********************************************
MinMaxScaler(): 55723012080.242775
***********************************************

*MinMaxScaler()*
***********************************************
Normalizer(): 8304789079624.974
***********************************************

***********************************************
MaxAbsScaler(): 8304789079624.974
***********************************************



In [12]:
mms = MinMaxScaler()
x_train[num_col] = mms.fit_transform(x_train[num_col])
x_test[num_col] = mms.fit_transform(x_test[num_col])
x_train

,squareMeters,hasYard,hasPool,cityCode,isNewBuilt,hasStormProtector,hasStorageRoom,cityPartRange_1,cityPartRange_2,cityPartRange_3,...,numBasement_E,numAttic_A,numAttic_B,numAttic_C,numAttic_D,numAttic_E,numGarage_A,numGarage_B,numGarage_C,numGarage_D
4852,0.332272,1,0,0.943184,0,1,0,1,0,0,...,0,0,0,0,1,0,0,0,0,1
315,0.995445,0,1,0.095335,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0
1314,0.999025,1,0,0.044151,1,0,0,0,0,0,...,0,0,0,1,0,0,1,0,0,0
22122,0.999597,1,0,0.028395,1,1,1,0,0,0,...,0,0,1,0,0,0,1,0,0,0
21573,0.994231,0,1,0.107261,0,0,1,0,0,0,...,1,0,0,0,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16247,0.995456,0,1,0.095225,1,0,1,0,0,1,...,0,1,0,0,0,0,0,0,1,0
2693,0.946587,0,1,0.322450,1,1,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
8076,0.999464,0,0,0.032751,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
20213,0.680740,1,0,0.732526,1,1,1,0,0,0,...,0,1,0,0,0,0,0,1,0,0


In [13]:
model = XGBRegressor()
model.fit(x_train, y_train)
predict = model.predict(x_test)
print(f'MSE: {mean_squared_error(y_test, predict)}')

MSE: 8304789079624.974


In [14]:
def evaluate_model(model):
    cv = ShuffleSplit(n_splits=10, test_size=0.3)
    score = cross_val_score(model, x_train, y_train, cv=cv, n_jobs=-1)
    return score

In [15]:
def get_model1():
    models = {}
    est = range(100, 501, 100)
    for e in est:
        models[str(e)] = XGBRegressor(n_estimators=e)
    return models
    
# models1 = get_model1()
# results, names = [], []
# for name, model in models1.items():
#     scores = evaluate_model(model)
#     results.append(scores)
#     names.append(name)
#     print(f'{name}: {np.mean(scores):.5f} ({np.std(scores):.5f})')

    
# plt.boxplot(results, labels=names)
# plt.show()

In [16]:
def get_model2():
    models = {}
    depth = range(3,11)
    for d in depth:
        models[str(d)] = XGBRegressor(n_estimators=100, max_depth=d)
    return models
    
# models2 = get_model2()
# results, names = [], []
# for name, model in models2.items():
#     scores = evaluate_model(model)
#     results.append(scores)
#     names.append(name)
#     print(f'{name}: {np.mean(scores):.5f} ({np.std(scores):.5f})')

    
# plt.boxplot(results, labels=names)
# plt.show()

In [17]:
def get_model2():
    models = {}
    depth = range(11,21)
    for d in depth:
        models[str(d)] = XGBRegressor(n_estimators=100, max_depth=d)
    return models
    
# models2 = get_model2()
# results, names = [], []
# for name, model in models2.items():
#     scores = evaluate_model(model)
#     results.append(scores)
#     names.append(name)
#     print(f'{name}: {np.mean(scores):.5f} ({np.std(scores):.5f})')

    
# plt.boxplot(results, labels=names)
# plt.show()

In [18]:
def get_model3():
    models = {}
    leaves = range(2,11)
    for l in leaves:
        models[str(l)] = XGBRegressor(n_estimators=100, max_depth=5, max_leaves=2**l)
    return models
    
# models3 = get_model3()
# results, names = [], []
# for name, model in models3.items():
#     scores = evaluate_model(model)
#     results.append(scores)
#     names.append(name)
#     print(f'{name}: {np.mean(scores):.5f} ({np.std(scores):.5f})')

    
# plt.boxplot(results, labels=names)
# plt.show()

In [19]:
def get_model3():
    models = {}
    leaves = range(5,31,5)
    for l in leaves:
        models[str(l)] = XGBRegressor(n_estimators=100, max_depth=5, max_leaves=2**l)
    return models
    
# models3 = get_model3()
# results, names = [], []
# for name, model in models3.items():
#     scores = evaluate_model(model)
#     results.append(scores)
#     names.append(name)
#     print(f'{name}: {np.mean(scores):.5f} ({np.std(scores):.5f})')

    
# plt.boxplot(results, labels=names)
# plt.show()

In [20]:
def get_model4():
    models = {}
    rate = [0.25,0.5,0.75, 1.0, 1.25, 1.5]
    for r in rate:
        models[str(r)] = XGBRegressor(n_estimators=100, max_depth=5, max_leaves=2**20, learning_rate=r)
    return models
    
# models4 = get_model4()
# results, names = [], []
# for name, model in models4.items():
#     scores = evaluate_model(model)
#     results.append(scores)
#     names.append(name)
#     print(f'{name}: {np.mean(scores):.5f} ({np.std(scores):.5f})')

    
# plt.boxplot(results, labels=names)
# plt.show()

In [21]:
def get_model4():
    models = {}
    rate = [0.05,0.1,0.15, 0.20, 0.25]
    for r in rate:
        models[str(r)] = XGBRegressor(n_estimators=100, max_depth=5, max_leaves=2**20, learning_rate=r)
    return models
    
# models4 = get_model4()
# results, names = [], []
# for name, model in models4.items():
#     scores = evaluate_model(model)
#     results.append(scores)
#     names.append(name)
#     print(f'{name}: {np.mean(scores):.5f} ({np.std(scores):.5f})')

    
# plt.boxplot(results, labels=names)
# plt.show()

In [22]:
def get_model5():
    models = {}
    jobs = range(2,11)
    for j in jobs:
        models[str(j)] = XGBRegressor(n_estimators=100, max_depth=5, max_leaves=2**20, learning_rate=0.05, n_jobs=j)
    return models
    
# models5 = get_model5()
# results, names = [], []
# for name, model in models5.items():
#     scores = evaluate_model(model)
#     results.append(scores)
#     names.append(name)
#     print(f'{name}: {np.mean(scores):.5f} ({np.std(scores):.5f})')

    
# plt.boxplot(results, labels=names)
# plt.show()

In [23]:
def get_model5():
    models = {}
    jobs = range(10,16)
    for j in jobs:
        models[str(j)] = XGBRegressor(n_estimators=100, max_depth=5, max_leaves=2**20, learning_rate=0.05, n_jobs=j)
    return models
    
# models5 = get_model5()
# results, names = [], []
# for name, model in models5.items():
#     scores = evaluate_model(model)
#     results.append(scores)
#     names.append(name)
#     print(f'{name}: {np.mean(scores):.5f} ({np.std(scores):.5f})')

    
# plt.boxplot(results, labels=names)
# plt.show()

In [24]:
def get_model6():
    models = {}
    weight = [0.75, 0.9,  1.0, 1.1, 1.25]
    for w in weight:
        models[str(w)] = XGBRegressor(n_estimators=100, max_depth=5, max_leaves=2**20, learning_rate=0.05, n_jobs=14, min_child_weight=w)
    return models
    
# models6 = get_model6()
# results, names = [], []
# for name, model in models6.items():
#     scores = evaluate_model(model)
#     results.append(scores)
#     names.append(name)
#     print(f'{name}: {np.mean(scores):.5f} ({np.std(scores):.5f})')

    
# plt.boxplot(results, labels=names)
# plt.show()

In [25]:
def get_model7():
    models = {}
    step = [5.25, 5.5,  5.75, 6.0, 6.25]
    for s in step:
        models[str(s)] = XGBRegressor(n_estimators=100, 
                                      max_depth=5, 
                                      max_leaves=2**20, 
                                      learning_rate=0.05, 
                                      n_jobs=14, 
                                      min_child_weight=1.1,
                                      max_delta_step = s)
    return models
    
# models7 = get_model7()
# results, names = [], []
# for name, model in models7.items():
#     scores = evaluate_model(model)
#     results.append(scores)
#     names.append(name)
#     print(f'{name}: {np.mean(scores):.5f} ({np.std(scores):.5f})')

    
# plt.boxplot(results, labels=names)
# plt.show()

In [26]:
model2 = XGBRegressor(n_estimators=100, max_depth=5, max_leaves=2**20, learning_rate=0.05, n_jobs=14, min_child_weight=1.1)
model2.fit(x_train, y_train)
predict2 = model2.predict(x_test)
print(f'MSE: {mean_squared_error(y_test, predict2)}')

MSE: 8226680043080.309


In [27]:
test = pd.read_csv('/kaggle/input/playground-series-s3e6/test.csv')
id_ = test.id
test = test.drop('id', axis=1)

In [28]:
test = prepare_data(test)
test = pd.get_dummies(data=test, columns=cat_num)
test.head()

,squareMeters,hasYard,hasPool,cityCode,isNewBuilt,hasStormProtector,hasStorageRoom,cityPartRange_1,cityPartRange_2,cityPartRange_3,...,numBasement_E,numAttic_A,numAttic_B,numAttic_C,numAttic_D,numAttic_E,numGarage_A,numGarage_B,numGarage_C,numGarage_D
0,47580.0,0,1,54830,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
1,62083.0,0,0,8576,1,1,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
2,90499.0,1,1,62454,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
3,16354.0,1,1,9262,1,1,1,0,0,0,...,1,0,0,1,0,0,0,0,0,1
4,67510.0,0,0,24112,1,1,1,0,0,1,...,0,0,1,0,0,0,0,0,1,0


In [29]:
test[num_col] = mms.fit_transform(test[num_col])
train[num_col] = mms.fit_transform(train[num_col])

In [30]:
model3 = XGBRegressor(n_estimators=100, max_depth=5, max_leaves=2**20, learning_rate=0.05, n_jobs=14, min_child_weight=1.1)
model3.fit(train, target)
predict3 = model3.predict(test)

In [31]:
sample_submission = pd.DataFrame(data={'id':id_, 'price':predict3})

In [32]:
sample_submission.to_csv('submission.csv', index=False)